# Data Science Prompt Engineer - Fine-Tuning with Unsloth on Colab

This notebook fine-tunes a 4-bit quantized LLM using Unsloth on Google Colab to generate Python code for data analysis prompts. It uses the `sahil2801/CodeAlpaca-20k` dataset filtered for Python examples.

## Install Dependencies

In [1]:
!pip install -U "triton==2.0.0"

  Using cached triton-2.0.0-1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.0 kB)
Using cached triton-2.0.0-1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.8.1 requires torch>=2.4.0, but you have torch 2.0.1 which is incompatible.
unsloth 2025.8.1 requires triton>=3.0.0; platform_system == "Linux", but you have triton 2.0.0 which is incompatible.
unsloth-zoo 2025.8.1 requires torch>=2.4.0, but you have torch 2.0.1 which is incompatible.
unsloth-zoo 2025.8.1 requires triton>=3.0.0; platform_system == "Linux", but you have triton 2.0.0 which is incompatible.
xformers 0.0.31.post1 requires torch==2.7.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.22.1 requires torch==2.7.1, but you have torch 2.0.1 which is incompatible.
torchaudio 2.6.0+c

In [2]:
!pip install unsloth
!pip install --no-deps xformers trl peft accelerate bitsandbytes datasets

  Using cached torch-2.8.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached triton-3.4.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cusolver_cu12-11.7.3.9

## Login to Hugging Face

In [3]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get("HuggingFace")  # Add your HF token in Colab Secrets
login(token=hf_token)

## Load Model with Unsloth

In [4]:
from unsloth import FastLanguageModel

max_seq_length = 2048
load_in_4bit = True

dtype = None  # Auto-detect dtype

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Mistral patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

## Load and Format Dataset

In [5]:
from datasets import load_dataset

dataset = load_dataset("sahil2801/CodeAlpaca-20k", split="train")

def filter_python(example):
    return "python" in example["output"].lower()

dataset = dataset.filter(filter_python)

README.md:   0%|          | 0.00/147 [00:00<?, ?B/s]

code_alpaca_20k.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/20022 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20022 [00:00<?, ? examples/s]

## Define Prompt Format

In [6]:
prompt_style = """You are an Experienced Data Science Assistant. Given an instruction, write the appropriate Python code.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples.get("input", [""] * len(instructions))
    outputs = examples["output"]
    texts = [prompt_style.format(inst, inp, out) + EOS_TOKEN for inst, inp, out in zip(instructions, inputs, outputs)]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

## Add PEFT and Trainer

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = False,
    random_state = 42,
    max_seq_length = max_seq_length,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,  # Increase for better results
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

Unsloth 2025.8.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Tokenizing ["text"]:   0%|          | 0/374 [00:00<?, ? examples/s]

## Train the Model

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 374 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zyrexan (zyrexan-cairo-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,1.329400
2,1.404800
3,1.429600
4,1.274500
5,1.328200
6,1.010500
7,0.743500
8,0.809000
9,0.755200
10,0.816500


Unsloth: Will smartly offload gradients to save VRAM!


## Save and Push to Hugging Face

In [9]:
new_model_name = "ZyrexAN/DataScience-PromptEngineer"

model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

model.push_to_hub(new_model_name, use_auth_token=True)
tokenizer.push_to_hub(new_model_name, use_auth_token=True)

README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pfkxzxf51/adapter_model.safetensors:   0%|          |  563kB /  168MB            

Saved model to https://huggingface.co/ZyrexAN/DataScience-PromptEngineer


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpm4qru9bo/tokenizer.model      : 100%|##########|  587kB /  587kB            

In [11]:
# Merge Adapter LoRA into the Base Model and Push to Hugging Face

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from huggingface_hub import notebook_login

base_model_id = "unsloth/mistral-7b-v0.3-bnb-4bit"  # The exact base you used!
adapter_repo = "ZyrexAN/DataScience-PromptEngineer"  # Your LoRA adapter repo
merged_repo = "ZyrexAN/DataScience-PromptEngineer-merged"  # Name for merged model repo

# Load base and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Load and merge adapter
model = PeftModel.from_pretrained(base_model, adapter_repo)
model = model.merge_and_unload()

# Save and push merged model
output_dir = "merged_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

model.push_to_hub(merged_repo)
tokenizer.push_to_hub(merged_repo)

print(f"✅ Successfully merged and pushed to {merged_repo}")


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp7lfv9moe/model.safetensors    :   0%|          | 30.0kB / 4.14GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp2mqg0pb1/tokenizer.model      : 100%|##########|  587kB /  587kB            

✅ Successfully merged and pushed to ZyrexAN/DataScience-PromptEngineer-merged


## Fin!
The model is ready now!